<a href="https://colab.research.google.com/github/alphabeen/ajou_multimodal/blob/main/ajou_multimodal_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 1) 필요한 패키지
!pip -q install nbformat

import nbformat as nbf
from copy import deepcopy
from google.colab import files
import os

# === 경로 설정: 현재 노트북을 file > Download 해서 /content/에 올렸다면 파일명 입력 ===
# 구글 드라이브에 있다면 /content/drive 를 마운트해서 경로를 지정해도 됩니다.
SRC = "/content/drive/MyDrive/Colab_Notebooks/ajou_multimodal.ipynb"
DST = "/content/ajou_multimodal_clean.ipynb"    # 정리본 파일명

nb = nbf.read(SRC, as_version=nbf.NO_CONVERT)

# 2) 노트북 metadata에서 widgets 제거
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# 3) 셀 출력에서 위젯 뷰/상태 MIME 제거
for cell in nb.cells:
    if cell.get("outputs"):
        new_outputs = []
        for out in cell["outputs"]:
            # outputs 중 'data'에 위젯 뷰가 들어있으면 제거
            if isinstance(out, dict) and "data" in out:
                data = deepcopy(out["data"])
                # 위젯 관련 MIME 제거
                data.pop("application/vnd.jupyter.widget-view+json", None)
                data.pop("application/vnd.jupyter.widget-state+json", None)
                # data가 비어도 다른 텍스트/이미지가 있으면 유지
                o2 = deepcopy(out)
                o2["data"] = data
                # 아무 것도 안 남은 display_data면 스킵
                if not data and o2.get("output_type") == "display_data":
                    continue
                new_outputs.append(o2)
            else:
                # stream/error 등은 그냥 유지
                new_outputs.append(out)
        cell["outputs"] = new_outputs

# 4) (선택) 실행번호 제거로 깔끔하게
for cell in nb.cells:
    if "execution_count" in cell:
        cell["execution_count"] = None

# 5) 저장
nbf.write(nb, DST)
print(f"Saved cleaned notebook to: {DST}")

# 6) 로컬로 다운로드 (원하면)
# files.download(DST)


Saved cleaned notebook to: /content/ajou_multimodal_clean.ipynb


In [ ]:
path = '/content/drive/MyDrive/deep_chal_multitask_dataset_sample.parquet'
df = pd.read_parquet(path)

df.head()

,input_type,task,input,output,question
0,image,captioning,https://i.pinimg.com/originals/c8/c9/76/c8c976...,\nThe image is a digital illustration of a fut...,
1,image,captioning,https://pulpcovers.com/wp-content/uploads/2022...,"\nThe image is the cover of a book titled ""A P...",
2,image,captioning,https://i.redd.it/079wfb6b6fec1.jpg,\nThe image is a black and white illustration ...,
3,image,captioning,https://pulpcovers.com/wp-content/uploads/2020...,"\nThe image is the cover of a book titled ""Det...",
4,image,captioning,https://pulpcovers.com/wp-content/uploads/2015...,"\nThe image is the cover of a book titled ""Lov...",


In [ ]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch

model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

processor = AutoProcessor.from_pretrained(model_id)

model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2160: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

In [ ]:
from PIL import Image
import requests, io
import torch

VISION_RESOLUTION = (224, 224)  # Qwen2.5-VL 기본 해상도

def preprocess_unified(example):
    """
    example: dict with keys
      input_type (image/text), task, input, question
    Returns dict containing exactly these four tensors:
      - input_ids          (LongTensor [seq_len])
      - attention_mask     (LongTensor [seq_len])
      - pixel_values       (FloatTensor [3,H,W])
      - image_grid_thw     (LongTensor or FloatTensor [T,H',W'])
    """
    # (A) 이미지 태스크
    if example["input_type"] == "image":
        img = (
            Image.open(requests.get(example["input"], stream=True).raw)
                 .convert("RGB")
        ) if isinstance(example["input"], str) else \
              Image.open(io.BytesIO(example["input"])).convert("RGB")
        model_inputs = processor(
            images=img,
            text=(
                f"<|Instruction|>{example['task']} "
                + (f"<|Question|>{example['question']} " if example['question'] else "")
                + "<|Response|>"
            ),
            return_tensors="pt"
        )

    # (B) 텍스트 태스크 — 반드시 dummy image 를 함께 넘겨줍니다.
    else:
        dummy_img = torch.zeros((1, 3, *VISION_RESOLUTION), dtype=torch.float32)
        model_inputs = processor(
            images=dummy_img,
            text=(
                f"<|Instruction|>{example['task']} "
                + f"<|Input|>{example['input']} "
                + (f"<|Question|>{example['question']} " if example['question'] else "")
                + "<|Response|>"
            ),
            return_tensors="pt"
        )

    # squeeze batch dim -> shape 맞춰줍니다.
    return {
        "input_ids":      model_inputs.input_ids     .squeeze(0),
        "attention_mask": model_inputs.attention_mask.squeeze(0),
        "pixel_values":   model_inputs.pixel_values   .squeeze(0),
        "image_grid_thw": model_inputs.image_grid_thw .squeeze(0),
    }


In [ ]:
import torch

device = model.device  # 이미 device_map="auto" 로 로드된 상태라고 가정

def unified_inference(example):
    # 1) 전처리 (batch 차원 제거된 텐서 네 개)
    inputs = preprocess_unified(example)

    # 2) batch 차원(=1) 붙이고 device 로 이동
    for k, v in inputs.items():
        inputs[k] = v.unsqueeze(0).to(device)

    # 3) 단 한 줄의 generate 호출 (분기 NO)
    with torch.no_grad():
        out_ids = model.generate(
            input_ids      = inputs["input_ids"],
            attention_mask = inputs["attention_mask"],
            pixel_values   = inputs["pixel_values"],
            image_grid_thw = inputs["image_grid_thw"],
            max_new_tokens = 128,
            temperature    = 0.7,
        )

    # 4) 디코딩
    return processor.decode(out_ids[0], skip_special_tokens=True)


In [ ]:
# 1) DataFrame에서 샘플 한 행 꺼내기
sample_row = df.iloc[0]

# 2) 필요한 키만 뽑아서 dict로 만들기
example = {
    "input_type": sample_row["input_type"],
    "task":       sample_row["task"],
    "input":      sample_row["input"],
    "question":   sample_row["question"],
}

# 3) 추론 실행
print(unified_inference(example))




ValueError: Image features and image tokens do not match: tokens: 0, features 391